# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` は使用しない
* 判定は **ID 基準**、表示は仕様どおりの列名と順序（`id, name, sex, salary`）

## 1) 問題

* `Salary.sex` の `'m'` / `'f'` を **単一の UPDATE 文**で入れ替える（中間テーブル・SELECT 不可）
* 入力テーブル例: `Salary(id INT PK, name VARCHAR, sex ENUM('m','f'), salary INT)`
* 出力仕様: `Salary` 全行で `sex` が `'m' <-> 'f'` に反転していること（他列は不変）

## 2) 最適解（単一クエリ）

> **CASE 一発更新**。ENUM でも文字列比較で問題なし。

```sql
UPDATE Salary
SET sex = CASE sex
            WHEN 'm' THEN 'f'
            ELSE 'm'
          END;

Runtime 231 ms
Beats 59.46%

```

* 単一トランザクションで全行を反転。
* `sex` が `'m'` / `'f'` 想定のため `ELSE 'm'` ではなく `ELSE 'm'|'f'` 判定の汎用性を残すために `ELSE 'm'` とせず **`ELSE 'm'` ではなく上記のように包括**（本テーブル前提では実質 `'f'` ）。

## 3) 代替解

> 関数ベースで分岐をさらに短く。いずれも **単一 UPDATE**。

```sql
-- A) IF 版（読みやすさ重視）
UPDATE Salary
SET sex = IF(sex = 'm', 'f', 'm');

Runtime 235 ms
Beats 51.52%

-- B) FIELD/ELT 版（対称性重視）
UPDATE Salary
SET sex = ELT(FIELD(sex, 'm','f'), 'f','m');

Runtime 238 ms
Beats 46.32%
```

## 4) 要点解説

* **ENUM でも文字列代入でOK**：`ENUM('m','f')` は `'m'` / `'f'` のいずれかのみ許容。CASE/IF の結果がそのいずれかなら整合。
* **中間テーブル・SELECT 不要**：全行同時に反転しても相互依存はなく、レース条件なし。
* **インデックス影響**：`sex` にインデックスがあっても UPDATE 自体は発生するが、全表更新のため恩恵は限定的。バッチ時間はテーブルサイズ依存。
* **NULL 取り扱い**：`NULL` を許可していない想定（問題文より）。許可される設計なら `CASE` に `WHEN sex IS NULL THEN NULL` を足す。

## 5) 計算量（概算）

* 全表更新で **O(N)**。I/O はページ数とログ設定（binlog, undo）に依存。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Salary テーブル] --> B[単一 UPDATE で sex を反転]
  B --> C[列: id name sex salary]
  C --> D[出力: sex が m/f 入れ替わり 他列不変]
```
結論：その差は誤差です。
この手の「全件同一列更新・2 値トグル」は I/O（ページ読み書き・undo/redo/binlog）支配で、`CASE` / `IF` / `ELT+FIELD` の式評価差はほぼ出ません。提示の 231–238ms の差は計測ノイズと見てよいです。

実務での“改善の余地”は以下の観点です（**クエリ自体はすでに最適級**）。

## 速さそのものを上げる余地（限定的）

* **式は読みやすさ優先で OK**
  `UPDATE Salary SET sex = IF(sex='m','f','m');` を推奨。最短・明快。速度差は出ません。
* （小ネタ）`ELT(FIELD(...))` や ASCII 計算でのトグルは書けますが、**可読性低下のデメリットが勝ち**。ベンチ差もほぼゼロです。

## 実運用の安全性・副作用を最適化

* **必要行だけ更新**（将来、混在があり得る場合）

```sql
UPDATE Salary
SET sex = IF(sex='m','f','m')
WHERE sex IN ('m','f');  -- 実質全件だが「無関係行」除外の保険

Runtime 229 ms
Beats 63.30%

```

変更不要行がある環境では I/O を減らせます。
* **大規模テーブルなら分割更新でロック/ログ圧を緩和**（総時間は増えるが可用性向上）

```sql
-- 例：PK でレンジ刻み
UPDATE Salary SET sex = IF(sex='m','f','m') WHERE id BETWEEN ? AND ?;
```
* **影響最小化の運用**
  トランザクションで囲む、ピーク外に実施、binlog/redo の容量監視、適切な `innodb_flush_log_at_trx_commit`・レプリカ遅延観測など。

## スキーマレベルでの抜本策（要変更可の場合）

* 性別を `ENUM('m','f')` ではなく **`TINYINT(1)` or `BIT(1)`** にすると
  将来のトグルは `UPDATE ... SET sex = 1 - sex;` と**最小計算・最小転送**で済みます（ただしアプリ側で 0/1 ↔ 表示文字のマッピングが必要）。

---

### 結局どれを使う？

可読性・保守性で **IF 版**がベストです。速度はもう頭打ちなので、**運用（ロック・ログ・時間帯）最適化**に寄せるのがビジネス的に賢い選択です。

